In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
from retrieve_data import train_data, validation_data, test_data

In [3]:
Xg_train, Xl_train, label_train = train_data()
Xg_valid, Xl_valid, label_valid = validation_data()
Xg_test, Xl_test, label_test = test_data()

2025-10-05 15:50:35.459053: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:390] TFRecordDataset `buffer_size` is unspecified, default to 262144
2025-10-05 15:50:35.562419: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-10-05 15:50:35.673009: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-10-05 15:50:35.900271: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-10-05 15:50:36.351502: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [4]:
# Add another dimension to be able to pass to convolutions
Xg_train = np.expand_dims(Xg_train, axis=-1)  
Xl_train = np.expand_dims(Xl_train, axis=-1)  

Xg_valid = np.expand_dims(Xg_valid, axis=-1)
Xl_valid = np.expand_dims(Xl_valid, axis=-1)

Xg_test = np.expand_dims(Xg_test, axis=-1)
Xl_test = np.expand_dims(Xl_test, axis=-1)
print(Xg_train.shape)

(12589, 2001, 1)


In [6]:
import keras_tuner as kt
from cnn_hyperparameterizable import build_cnn_global_local_model

In [11]:
tuner = kt.Hyperband(
    build_cnn_global_local_model,
    objective=kt.Objective("val_recall", direction='max'),
    max_epochs = 30,
    factor = 3,
    hyperband_iterations=2,
    seed=42,
    directory='hyper_tuning',
    project_name='exoplanet_challenge',
    overwrite=False,
    executions_per_trial=1
)

In [9]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_recall",
    mode='max',
    patience=5,
    restore_best_weights=True,
    verbose=1
)
reduce_lr_on_plateau = keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

checkpointing = keras.callbacks.ModelCheckpoint(
    filepath="best_exoplanet_cnn.keras",
    monitor="val_recall",
    mode="max",
    save_best_only=True,
    verbose=1
)

tensorboarding = keras.callbacks.TensorBoard(
    log_dir="logs/exoplanet_cnn",
    histogram_freq=1
)

logging = keras.callbacks.CSVLogger("training_metrics.csv", append=False)

callbacks = [early_stopping, reduce_lr_on_plateau, checkpointing, tensorboarding, logging]

In [13]:
tuner.search(
    x={"global_input" : Xg_train, 'local_input' : Xl_train},
    y=label_train,
    validation_data=({"global_input" : Xg_valid, 'local_input' : Xl_valid}, label_valid),
    epochs=50,
    callbacks = callbacks
)

Trial 180 Complete [00h 00m 15s]
val_recall: 0.9267605543136597

Best val_recall So Far: 0.9577465057373047
Total elapsed time: 00h 50m 13s


In [14]:
best_model = tuner.get_best_models()[0]

/Users/swebber/pythonvenvs/quantumcomet/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 42 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [15]:
best_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ global_input        │ (None, 2001, 1)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ local_input         │ (None, 201, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 2001, 4)   │         24 │ global_input[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 201, 8)    │         48 │ local_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d       │ (None, 1000, 4)   │          0 │ conv1d[0][0]      │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_3     │ (None, 100, 8)    │          0 │ conv1d_3[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 1000, 8)   │        168 │ max_pooling1d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_4 (Conv1D)   │ (None, 100, 16)   │        656 │ max_pooling1d_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_1     │ (None, 500, 8)    │          0 │ conv1d_1[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_4     │ (None, 50, 16)    │          0 │ conv1d_4[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 500, 12)   │        492 │ max_pooling1d_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_5 (Conv1D)   │ (None, 50, 24)    │      1,944 │ max_pooling1d_4[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_2     │ (None, 250, 12)   │          0 │ conv1d_2[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_5     │ (None, 25, 24)    │          0 │ conv1d_5[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 3000)      │          0 │ max_pooling1d_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 600)       │          0 │ max_pooling1d_5[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │    192,064 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 16)        │      9,616 │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 64)        │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 16)        │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 80)        │          0 │ dropout[0][0],  

 Total params: 210,261 (821.33 KB)

 Trainable params: 210,261 (821.33 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
best_model.evaluate({'global_input' : Xg_test, 'local_input': Xl_test}, y=label_test)

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9422 - loss: 0.1398 - precision: 0.8257 - recall: 0.9472  


[0.13982826471328735,
 0.9421855211257935,
 0.9472222328186035,
 0.8256658315658569]